In [3]:
# Create the stops_geo layer

def create_stops_geo(input_file_path_string, spatial_ref):
    stop_times = input_file_path_string + '\stop_times.txt'
    stops = input_file_path_string + '\stops.txt'
    stop_times_geo = input_file_path_string + '\stop_times_geo.csv'
    unique_identifier_field = 'stop_id'
    data_dict_file2 = {}
    merged_data = []

    with open(stops, 'r', newline='') as file2:
        csvreader = csv.DictReader(file2)
        for row in csvreader:
            stop_id = row[unique_identifier_field]
            data_dict_file2[stop_id] = {key: row[key] for key in row if key != unique_identifier_field}

    with open(stop_times, 'r', newline='') as file1:
        csvreader = csv.DictReader(file1)
        for row in csvreader:
            stop_id = row[unique_identifier_field]
            data_from_file2 = data_dict_file2.get(stop_id, {})
            row.update(data_from_file2)
            merged_data.append(row)

    with open(stop_times_geo, 'w', newline='') as merged_file:
        fieldnames = merged_data[0].keys() 
        writer = csv.DictWriter(merged_file, fieldnames = fieldnames)
        writer.writeheader()
        for row in merged_data:
            writer.writerow(row)
        
    output_coordinate_system = arcpy.SpatialReference(spatial_ref)
    out_feature_class = 'Stops_Geo'
    output_projected_fc = 'Stops_Geo_Projected'
    arcpy.management.XYTableToPoint(stop_times_geo, out_feature_class, 'stop_lon', 'stop_lat')
    arcpy.management.Project(out_feature_class, output_projected_fc, output_coordinate_system)
    
#create_stops_geo(r'C:\School\GIS 5571\Local\Final Project Data\Data\GTFS', 26915)

In [4]:
# Remove unwanted service days and providers

def refine_calendar_service(stops_geo, sql_query):
    arcpy.SelectLayerByAttribute_management(stops_geo, "NEW_SELECTION", sql_query)
    arcpy.SelectLayerByAttribute_management(stops_geo, "SWITCH_SELECTION")
    arcpy.DeleteFeatures_management(stops_geo)
    
#refine_calendar_service("Stops_Geo_Projected", "trip_id LIKE '%RAIL-Weekday-01%' Or trip_id LIKE '%NS-Weekday-02%' Or trip_id LIKE '%BUS-Weekday-01%'")

In [5]:
# Code to add the route_id to the stops_geo_projected with the trips.csv table

def add_route_ids(stops_geo, GTFS_path):
        table_file_path = GTFS_path + "\\" + 'trips.txt'
        aprx = arcpy.mp.ArcGISProject("CURRENT")
        m = aprx.listMaps()[0]
        t = m.addDataFromPath(table_file_path)
        aprx.save()
        join_table = 'trips.txt'
        field = 'trip_id'
        
        arcpy.AddField_management(stops_geo, "route", "LONG", field_length=50)
        arcpy.AddJoin_management(stops_geo, field, join_table, field)
        arcpy.CalculateField_management(stops_geo, "route", "!trips.txt.route_id!", "PYTHON3", "", "LONG")
        arcpy.RemoveJoin_management(stops_geo, "trips.txt")
            
#add_route_ids("Stops_Geo_Projected", r'C:\School\GIS 5571\Local\Final Project Data\Data\GTFS')

In [6]:
# Here the stops_geo would be the Stops_Geo and in_stops is the layer of stop analysis

def set_buffer_and_fields(stops_geo, in_stops, walk_speed, max_walk_time):
    in_layer = stops_geo
    select_features = in_stops
    search_distance = f'{walk_speed * max_walk_time} Meters'
    arcpy.management.SelectLayerByLocation(in_layer, "INTERSECT", select_features, search_distance)
    
    output_feature_class = "Stops_Geo_Buffer"
    arcpy.CopyFeatures_management(in_layer, output_feature_class)
    arcpy.SelectLayerByAttribute_management(stops_geo, "CLEAR_SELECTION")
    arcpy.AddField_management(output_feature_class, 'latest_departure', "TIMEONLY")
            
#set_buffer_and_fields("Stops_Geo_Projected", "In_Stops", 83.33, 10)

In [7]:
# Use an existing network dataset to calculate the walk distance in the network

def calc_walk_dist(stops_geo, in_stops, network_dataset):
    output_class = "Stops_Geo_Buffer_Dissolve"
    arcpy.management.Dissolve(stops_geo, output_class, "stop_id")
    arcpy.AddField_management(output_class, "walk_distance", "DOUBLE")
    
    arcpy.na.MakeClosestFacilityLayer(network_dataset, "ClosestFacility", "Length")
    arcpy.na.AddLocations("ClosestFacility", "Facilities", in_stops)
    arcpy.na.AddLocations("ClosestFacility", "Incidents", output_class)
    arcpy.na.Solve("ClosestFacility")
    
    in_table = r"ClosestFacility\Routes"
    with arcpy.da.UpdateCursor(output_class, ["OBJECTID", "walk_distance"]) as update_cursor:
        for row in update_cursor:
            object_id = row[0]
            where_clause = f"ObjectID = {object_id}"
            with arcpy.da.SearchCursor(in_table, "Total_Length", where_clause) as search_cursor:
                for search_row in search_cursor:
                    row[1] = search_row[0]
            update_cursor.updateRow(row)
    
    arcpy.management.Delete("ClosestFacility")
    
    field = "stop_id"
    arcpy.AddField_management(stops_geo, "walk_distance", "DOUBLE")
    arcpy.AddJoin_management(stops_geo, field, output_class, field)
    arcpy.CalculateField_management(stops_geo, "walk_distance", "!Stops_Geo_Buffer_Dissolve.walk_distance!", "PYTHON3", "", "DOUBLE")
    arcpy.RemoveJoin_management(stops_geo, output_class)
    
    arcpy.management.Delete(output_class)

    
#calc_walk_dist("Stops_Geo_Buffer", "In_Stops", r'C:\School\GIS 5571\Local\Final Project Data\Data\Network Dataset Data\Existing_Network.gdb\NetworkFeatureDataset\TransitNetwork_ND')

In [8]:
# This function removes values where the time is about 24 hours

def remove_24_plus(stops_geo):
    sql_query = "departure_time > '23:59:00'"
    arcpy.SelectLayerByAttribute_management(stops_geo, "NEW_SELECTION", sql_query)
    arcpy.DeleteFeatures_management(stops_geo)
    
#remove_24_plus("Stops_Geo_Buffer")

In [9]:
# Here we calculate the latest departure values and export the matrix of reachable connection in the threshold

def calc_fields(in_stops, stops_geo, transfer_threshold, walk_speed):
    expression = "calc_latest_dept(!arrival_time!, !walk_distance!)"
    code_block = f"""
from datetime import datetime, timedelta

def calc_latest_dept(departure_time, distance):
    walking_time = timedelta(minutes = distance / {walk_speed})
    departure_datetime = datetime.strptime(departure_time, '%H:%M:%S')
    latest_departure_time = departure_datetime - walking_time
    return str(latest_departure_time)"""
    
    arcpy.CalculateField_management(stops_geo, "latest_departure", expression, "PYTHON3", code_block)
    
    arrival_time_list = []
    with arcpy.da.SearchCursor(in_stops, "arrival_time") as cursor:
        for row in cursor:
            arrival_time_list.append(row[0])
                        
    index = 1
    threshold = f'0:{transfer_threshold}:00'
    for time in arrival_time_list:
        expression = f"calc_wait_threshold(!latest_departure!, !arrival_time!, '{threshold}')"
        code_block = f"""
from datetime import datetime

def calc_wait_threshold(latest_dept, arrival, threshold_time):
    station_arrival_time = datetime.strptime('{time}', '%H:%M:%S')
    latest_departure = datetime.strptime(latest_dept, '%H:%M:%S')
    transfer_arrival_time = datetime.strptime(arrival, '%H:%M:%S')
    waiting_time = latest_departure - station_arrival_time
    transfer_buffer = transfer_arrival_time - station_arrival_time
    if (str(transfer_buffer) < threshold_time and str(waiting_time) > '0:00:00'):
        return 'Y'
    else:
        return 'N'"""
        
        input_field = f"In_Stops_{index}"
        arcpy.CalculateField_management(stops_geo, input_field, expression, "PYTHON3", code_block)
        index = index + 1

#calc_fields("In_Stops", "Stops_Geo_Buffer", 20, 83.33)

In [10]:
# Now we export the reachable connections based on highest stop_sequence and for stop_sequences that aren't in the last 3 stops along a trip

def export_connections(stops_geo_buffer, stops_geo, in_stops):
    in_stop_num = int((arcpy.GetCount_management(in_stops)).getOutput(0))
    while in_stop_num > 0:
        trip_ids = set()
        with arcpy.da.SearchCursor(stops_geo_buffer, ["trip_id", f"In_Stops_{in_stop_num}"]) as cursor:
            for row in cursor:
                if row[1] == 'Y':
                    trip_ids.add(row[0])

        unique_trip_ids = list(trip_ids)
        min_distances = {}
        for trip_id in unique_trip_ids:
            min_distance = float('inf')
            min_stop_sequence = -1
            with arcpy.da.SearchCursor(stops_geo_buffer, ["walk_distance", "stop_sequence"], f"trip_id = '{trip_id}' and In_Stops_{in_stop_num} = 'Y'") as cursor:
                for row in cursor:
                    if row[0] < min_distance:
                        min_distance = row[0]
                        min_stop_sequence = row[1]
            min_distances[trip_id] = {'distance': min_distance, 'min_stop_sequence': min_stop_sequence}
    
        out_path = r"C:\School\GEOG 8294\Final Project Data Testing\Final_Scratchspace\Final_Scratchspace.gdb"
        output_table = f"In_Stop_{in_stop_num}_Connection_Min"
        arcpy.CreateTable_management(out_path, output_table)
        arcpy.AddField_management(output_table, 'trip_id', "TEXT")
        arcpy.AddField_management(output_table, 'stop_sequence', "LONG")
        
        for trip_id, data in min_distances.items():
            max_stop_sequence = -1
            with arcpy.da.SearchCursor(stops_geo, ["stop_sequence"], f"trip_id = '{trip_id}'") as cursor:
                for row in cursor:
                    max_stop_sequence = max(max_stop_sequence, row[0])

            difference = max_stop_sequence - data['min_stop_sequence']
            if difference > 3:
                return_stop = {'trip_id': trip_id, 'stop_sequence': data}
                loop_distance = data['distance']
                current_stop_sequence = data['min_stop_sequence']
                while True:
                    with arcpy.da.SearchCursor(stops_geo_buffer, ["walk_distance"], f"trip_id = '{trip_id}' AND stop_sequence = {current_stop_sequence}") as cursor:
                        for row in cursor:
                            if row[0] < loop_distance + 50:
                                return_stop = {'trip_id': trip_id, 'stop_sequence': current_stop_sequence}
                    current_stop_sequence += 1
                    if current_stop_sequence > max_stop_sequence:
                        break
                        
                with arcpy.da.InsertCursor(output_table, ["trip_id", "stop_sequence"]) as cursor:
                    cursor.insertRow((return_stop['trip_id'], return_stop['stop_sequence']))
                    
        in_stop_num = in_stop_num - 1
        
#export_connections("Stops_Geo_Buffer", "Stops_Geo_Projected", "In_Stops")

In [11]:
# Add the remaining fields for the output analysis

def add_fields(stops_geo, in_stops):
    in_stop_num = int((arcpy.GetCount_management(in_stops)).getOutput(0))
    while in_stop_num > 0:
        out_name = f"In_Stop_{in_stop_num}_Connection_Min"
        fields_to_extract = ["route", "stop_id", "arrival_time", "latest_departure"]

        for field in fields_to_extract:
            arcpy.AddField_management(out_name, field, "TEXT")

        with arcpy.da.UpdateCursor(out_name, ["trip_id", "stop_sequence"] + fields_to_extract) as update_cursor:
            for row in update_cursor:
                trip_id = row[0]
                current_stop_sequence = row[1]

                where_clause = f"trip_id = '{trip_id}' AND stop_sequence = {current_stop_sequence}"
                with arcpy.da.SearchCursor(stops_geo, fields_to_extract, where_clause) as search_cursor:
                    for search_row in search_cursor:
                        
                        for idx, field in enumerate(fields_to_extract):
                            row[idx + 2] = str(search_row[idx])

                update_cursor.updateRow(row)

        in_stop_num = in_stop_num - 1
        
        arrival_time_list = []
        with arcpy.da.SearchCursor(in_stops, "arrival_time") as cursor:
            for row in cursor:
                arrival_time_list.append(row[0])

        arcpy.AddField_management(out_name, 'wait_time', "TEXT")
        expression = "calc_wait_time(!latest_departure!)"
        code_block = f"""
from datetime import datetime        

def calc_wait_time(latest_dept):
    station_arrival_time = datetime.strptime('{arrival_time_list[in_stop_num]}', '%H:%M:%S')
    latest_departure = datetime.strptime(latest_dept, '%H:%M:%S')
    waiting_time = latest_departure - station_arrival_time
    return str(waiting_time)
"""
        arcpy.CalculateField_management(out_name, "wait_time", expression, "PYTHON3", code_block)
    
#add_fields("Stops_Geo_Buffer", "In_Stops")

In [12]:
# user inputs for the main function

import csv
import zipfile
import requests
from datetime import datetime, timedelta

arcpy.env.workspace = r'C:\School\GEOG 8294\Final Project Data Testing\Final_Scratchspace\Final_Scratchspace.gdb'

url = 'https://svc.metrotransit.org/mtgtfs/archive/gtfs20230923.zip'
GTFS_path = r'C:\School\GIS 5571\Local\Final Project Data\Data\GTFS'
spatial_ref = 26915 # spatial reference system
sql_query = "trip_id LIKE '%RAIL-Weekday-01%' Or trip_id LIKE '%NS-Weekday-02%' Or trip_id LIKE '%BUS-Weekday-01%'" # SQL query used to filter unwanted calendar service
in_stops = "In_Stops" # name of the input arrival stops layer
walk_speed = 83.33 # used to calculate distance querried for possible transfer stops with max_walk_time
max_walk_time = 10 # used to calculate distance querried for possible transfer stops with walk_speed
network_dataset = r'C:\School\GIS 5571\Local\Final Project Data\Data\Network Dataset Data\Existing_Network.gdb\NetworkFeatureDataset\TransitNetwork_ND' # input public transit network dataset for the walk distance
transfer_threshold = 20 # total amount of time allowed between arrival time from input stop and departure of the transfer service

In [13]:
def main(url, GTFS_path, spatial_ref, sql_query, in_stops, walk_speed, max_walk_time, network_dataset, transfer_threshold):
    create_stops_geo(GTFS_path, spatial_ref)
    refine_calendar_service("Stops_Geo_Projected", sql_query)
    add_route_ids("Stops_Geo_Projected", GTFS_path)
    set_buffer_and_fields("Stops_Geo_Projected", in_stops, walk_speed, max_walk_time)
    calc_walk_dist("Stops_Geo_Buffer", in_stops, network_dataset)
    remove_24_plus("Stops_Geo_Buffer")
    calc_fields(in_stops, "Stops_Geo_Buffer", transfer_threshold, walk_speed)
    export_connections("Stops_Geo_Buffer", "Stops_Geo_Projected", in_stops)
    add_fields("Stops_Geo_Buffer", in_stops)
    
main(url, GTFS_path, spatial_ref, sql_query, in_stops, walk_speed, max_walk_time, network_dataset, transfer_threshold)